In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

In [2]:
stimuli = pd.read_excel('../../stimuli_specificity_complete.xlsx')[['set_nr','specificity','word1','word2']]
stimuli

,set_nr,specificity,word1,word2
0,1,low,mghljkqr,alloy
1,1,mid,brown,alloy
2,1,high,vpbyzkgrf,bronze
3,2,low,vpbyzkgrf,amphibian
4,2,mid,hopping,amphibian
...,...,...,...,...
355,119,mid,violent,weather
356,119,high,xkq,storm
357,120,low,kmlsdjtw,wine
358,120,mid,sparkling,wine


Extract specificity and items 

In [3]:
high = stimuli.query('specificity == "high"')[['set_nr','specificity','word2']].rename(columns={'word2':'option'})
mid = stimuli.query('specificity == "mid"')[['set_nr','specificity','word1','word2']]
mid['phrase'] = mid['word1'] + ' ' + mid['word2']
mid = mid[['set_nr','specificity','phrase']].rename(columns={'phrase':'option'})
low = stimuli.query('specificity == "low"')[['set_nr','specificity','word2']].rename(columns={'word2':'option'})
stimuli = pd.concat([low, mid, high])
stimuli

,set_nr,specificity,option
0,1,low,alloy
3,2,low,amphibian
6,3,low,animal
9,4,low,bedding
12,5,low,bird
...,...,...,...
347,116,high,submarine
350,117,high,tank
353,118,high,tractor
356,119,high,storm


Build pairs of options

In [4]:
options_new = []
specificity_new = []
set_nr_new = []
stimuli_shuffled = shuffle(stimuli, random_state=42) # shuffle everything 
for i_set in stimuli_shuffled.set_nr.unique():
    stimuli_subset = stimuli_shuffled.query('set_nr == @i_set').reset_index(drop=True)
    for combination in [[0,1],[1,2],[0,2]]:
        options_new.append(stimuli_subset.iloc[combination].option.tolist())
        specificity_new.append(stimuli_subset.iloc[combination].specificity.tolist())
        set_nr_new.append(i_set)
options_new = np.array(options_new).transpose()
stimuli_new = pd.DataFrame({'set_nr': set_nr_new,
                            'specificity': specificity_new,
                            'option1': options_new[0],
                            'option2': options_new[1]})
stimuli_new = stimuli_new.sort_values(by='set_nr').reset_index(drop=True)
stimuli_new['item_nr'] = stimuli_new.index + 1
stimuli_new = stimuli_new[stimuli_new.columns.tolist()[-1:] + stimuli_new.columns.tolist()[:-1]]
stimuli_new

,item_nr,set_nr,specificity,option1,option2
0,1,1,"[high, low]",bronze,alloy
1,2,1,"[mid, low]",brown alloy,alloy
2,3,1,"[high, mid]",bronze,brown alloy
3,4,2,"[low, mid]",amphibian,hopping amphibian
4,5,2,"[low, high]",amphibian,frog
...,...,...,...,...,...
355,356,119,"[low, mid]",weather,violent weather
356,357,119,"[high, low]",storm,weather
357,358,120,"[mid, high]",sparkling wine,champagne
358,359,120,"[low, high]",wine,champagne


Make two gorilla spreadsheets: counterbalancing of choosing the less or more specific item in a pair.

In [5]:
specificity_mapping = {'low':1, 'mid':2, 'high':3}
material = stimuli_new.copy()

for choose in ['less','more']:
    correct_answer = []
    for row in stimuli_new.iterrows():
        row = row[1]
        if choose == 'less':
            if specificity_mapping[row.specificity[0]] < specificity_mapping[row.specificity[1]]:
                correct_answer.append('left')
            else:
                correct_answer.append('right')
        elif choose == 'more':
            if specificity_mapping[row.specificity[0]] < specificity_mapping[row.specificity[1]]:
                correct_answer.append('right')
            else:
                correct_answer.append('left')
    material['correct_answer'] = correct_answer

    material = shuffle(material, random_state=42)

    # create dataframe 
    spreadsheet = pd.DataFrame(columns=['display','randomise_trials','option1','option2','correct_answer','condition_choose'])

    # create display
    list_displays = ['instructions_practice',
                     'trial_practice','trial_practice','trial_practice',
                     'trial_practice','trial_practice','trial_practice',
                     'trial_practice','trial_practice','trial_practice',
                     'repeat_practice',
                     'instructions_experimental']
    list_displays = list_displays + ['trial_experimental'] * 360 + ['end']
    spreadsheet['display'] = list_displays

    # create randomisation
    list_randomisation = ['']
    list_randomisation = list_randomisation + ['0']*9 # randomise 9 practice trials 
    list_randomisation = list_randomisation + ['','']
    list_randomisation = list_randomisation + ['1']*360 # randomise all 360 experimental trials
    spreadsheet['randomise_trials'] = list_randomisation + [''] 


    # create option1
    list_option1 = ['',
                    'white pigeon','bird','elegant bird',
                    'van','vehicle','Toyota',
                    'plant','lily','palm',
                    '','']
    list_option1 = list_option1 + material['option1'].tolist() + ['end']
    spreadsheet['option1'] = list_option1

    # create option2
    list_option2 = ['',
                    'pigeon','crow','swan',
                    'car','Audi','automobile',
                    'cactus','flower','tree',
                    '','']
    list_option2 = list_option2 + material['option2'].tolist() + ['end']
    spreadsheet['option2'] = list_option2

    # create answer
    if choose == 'less':
        list_answer = ['',
                        'right','left','left',
                        'right','left','right',
                        'left','right','right',
                        '','']
    elif choose == 'more':
        list_answer = ['',
                        'left','right','right',
                        'left','right','left',
                        'right','left','left',
                        '','']
    list_answer = list_answer + material['correct_answer'].tolist() + ['end']
    spreadsheet['correct_answer'] = list_answer

    spreadsheet['condition_choose'] = choose
    spreadsheet.to_csv(f'spreadsheet_gorilla_choose-{choose}.csv', index=False)